<a href="https://colab.research.google.com/github/nannaphatsirison/nannaphatsirison/blob/main/Optimization%20Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Set up of the problem**

1) A refinery produces three grades of gasoline using variable combinations of three types of crude oil as feed-stocks. \\
2) There is a conversion cost of $4 dollars to turn on barrel of crude into one barrel of gasoline. \\
3)The refinery has to meet the daily demand of each gasoline type but is also limited in the total number of gasoline barrels it can produce daily \\
4) Each type of gasoline has minimum octane limits as well as maximum sulfur content limits. \\
5) The refinery also has the option to market is products through an advertising campaign that can increase daily demand for each gasoline type. \\

**Objective:** Build an optimization model that maximizes profits of the refinery and helps the refinery decide how much it should spend on advertising to increase demand.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

In [ ]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('glpk')

In [ ]:
#Initialize Model
model=ConcreteModel()

#Defining Sets
model.G=Set(initialize=['G1','G2','G3'])  #set of gasoline types running from 1 to 3
model.C=Set(initialize=['C1','C2','C3'])  #set of crude types running from 1 to 3

#Defining Parameters
model.P=Param(model.C,initialize={'C1':70,'C2':62,'C3':57})        #purchase price for each crude type
model.S=Param(model.G,initialize={'G1':79,'G2':73,'G3':68})        #sale price for each gasoline type

model.D=Param(model.G,initialize={'G1':2200,'G2':3750,'G3':4200})  #forecasted demand for each gasoline type
model.E=Param(model.C,initialize={'C1':3500,'C2':5000,'C3':6000})  #purchase limit for each crude type 

model.H=Param(model.G,initialize={'G1':91,'G2':89,'G3':87})        #minimum octane rating for each gasoline type
model.J=Param(model.C,initialize={'C1':93,'C2':89,'C3':85})        #octane rating for each crude type

model.K=Param(model.G,initialize={'G1':0.8,'G2':1.4,'G3':1.8})        #maximum sulfur content for each gasoline type (%)
model.M=Param(model.C,initialize={'C1':0.5,'C2':1.7,'C3':2.1})        #sulfure content for each crude type (%)

#Add dec variables
model.X=Var(model.G,model.C,domain=NonNegativeReals)            # number of barrels of crude type c converted to gasoline type g
model.Y=Var(model.G, domain = NonNegativeReals)                 # how much money ($) is spent towards advertising for gasoline type g

In [ ]:
#Add obj func
def obj_profit(model):
    return sum(sum((model.S[g]-model.P[c]-4)*model.X[g,c]for g in model.G) for c in model.C) -(sum(1*model.Y[g] for g in model.G))
model.profit=Objective(sense=maximize,rule=obj_profit)

#Checking objetive function
print(model.profit.expr)

    'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
5*X[G1,C1] - X[G2,C1] - 6*X[G3,C1] + 13*X[G1,C2] + 7*X[G2,C2] + 2*X[G3,C2] + 18*X[G1,C3] + 12*X[G2,C3] + 7*X[G3,C3] - (Y[G1] + Y[G2] + Y[G3])


In [ ]:
#market demand constraint for all gasoline types
def meet_demand(model,g):
    return sum(model.X[g,c] for c in model.C) == model.D[g] + 10*model.Y[g]
model.demand=Constraint(model.G,rule=meet_demand)

#Checking added constraints
for g in model.G:
    print(model.demand[g].expr)


    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
X[G1,C1] + X[G1,C2] + X[G1,C3]  ==  2200 + 10*Y[G1]
X[G2,C1] + X[G2,C2] + X[G2,C3]  ==  3750 + 10*Y[G2]
X[G3,C1] + X[G3,C2] + X[G3,C3]  ==  4200 + 10*Y[G3]


In [ ]:
# crude purchase limits for all crude types
def purchase_limit(model,c):
    return sum(model.X[g,c] for g in model.G) <= model.E[c]
model.purchaselim=Constraint(model.C,rule=purchase_limit)

#Checking added constraints
for c in model.C:
    print(model.purchaselim[c].expr)


    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
X[G1,C1] + X[G2,C1] + X[G3,C1]  <=  3500
X[G1,C2] + X[G2,C2] + X[G3,C2]  <=  5000
X[G1,C3] + X[G2,C3] + X[G3,C3]  <=  6000


In [ ]:
# gasoline octane constraint
def octane_min(model,g):
    return sum(model.J[c]*model.X[g,c] for c in model.C) >= model.H[g]*sum(model.X[g,c] for c in model.C)
model.octanegasoline=Constraint(model.G,rule=octane_min)

#Checking added constraints
for g in model.G:
    print(model.octanegasoline[g].expr)


    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
91*(X[G1,C1] + X[G1,C2] + X[G1,C3])  <=  93*X[G1,C1] + 89*X[G1,C2] + 85*X[G1,C3]
89*(X[G2,C1] + X[G2,C2] + X[G2,C3])  <=  93*X[G2,C1] + 89*X[G2,C2] + 85*X[G2,C3]
87*(X[G3,C1] + X[G3,C2] + X[G3,C3])  <=  93*X[G3,C1] + 89*X[G3,C2] + 85*X[G3,C3]


In [ ]:
# sulfur content constraint
def octane_max(model,g):
    return sum(model.M[c]*model.X[g,c] for c in model.C) <= model.K[g]*sum(model.X[g,c] for c in model.C)
model.octanegasoline=Constraint(model.G,rule=octane_max)

#Checking added constraints
for g in model.G:
    print(model.octanegasoline[g].expr)


    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
0.5*X[G1,C1] + 1.7*X[G1,C2] + 2.1*X[G1,C3]  <=  0.8*(X[G1,C1] + X[G1,C2] + X[G1,C3])
0.5*X[G2,C1] + 1.7*X[G2,C2] + 2.1*X[G2,C3]  <=  1.4*(X[G2,C1] + X[G2,C2] + X[G2,C3])
0.5*X[G3,C1] + 1.7*X[G3,C2] + 2.1*X[G3,C3]  <=  1.8*(X[G3,C1] + X[G3,C2] + X[G3,C3])


In [ ]:
#Solve model
results = opt.solve(model)

#Print results
print("Profit =",model.profit())
for g in model.G:
  for c in model.C:
    print(model.X[g,c],model.X[g,c].value)
    print(model.Y[g],model.Y[g].value)

Profit = 63165.0
X[G1,C1] 1787.5
Y[G1] 0.0
X[G1,C2] 0.0
Y[G1] 0.0
X[G1,C3] 412.5
Y[G1] 0.0
X[G2,C1] 937.5
Y[G2] 0.0
X[G2,C2] 2812.5
Y[G2] 0.0
X[G2,C3] 0.0
Y[G2] 0.0
X[G3,C1] 775.0
Y[G3] 285.0
X[G3,C2] 2187.5
Y[G3] 285.0
X[G3,C3] 4087.5
Y[G3] 285.0


In [ ]:
if results.solver.status == 'ok':
    model.pprint()

3 Set Declarations
    C : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'C1', 'C2', 'C3'}
    G : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'G1', 'G2', 'G3'}
    X_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    G*C :    9 : {('G1', 'C1'), ('G1', 'C2'), ('G1', 'C3'), ('G2', 'C1'), ('G2', 'C2'), ('G2', 'C3'), ('G3', 'C1'), ('G3', 'C2'), ('G3', 'C3')}

8 Param Declarations
    D : Size=3, Index=G, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :  2200
         G2 :  3750
         G3 :  4200
    E : Size=3, Index=C, Domain=Any, Default=None, Mutable=False
        Key : Value
         C1 :  3500
         C2 :  5000
         C3 :  6000
    H : Size=3, Index=G, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :    91
  

In [ ]:
# GETTING SHADOW PRICES
model.dual=Suffix(direction=Suffix.IMPORT)

In [ ]:
#getting shadow price
results = opt.solve(model)
print ("Shadow Prices")
model.dual.pprint()

Shadow Prices
dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key                : Value
            demand[G1] : -11.9
            demand[G2] :  -4.1
            demand[G3] :   0.1
    octanegasoline[G1] :  23.0
    octanegasoline[G2] :  23.0
    octanegasoline[G3] :  23.0
       purchaselim[C1] :  23.8
       purchaselim[C2] :   4.2
       purchaselim[C3] :   0.0
